## optimize the parameter of model 

 . we use 5000 picture for training model , set 3500 picture as training data and then each half of data as valid data and testing data 

In [ ]:
import cv2 
import mysql.connector as con 
import time 
from datetime import datetime

url = "https://cctvn.freeway.gov.tw/abs2mjpg/bmjpg?camera=13380"

# connect mysql 

cursor = con.connect(
    host = "172.26.176.1",
    user = 'ruizhong' ,
    password = "123456",
    database = "the_volume_of_car"
)

cur = cursor.cursor()

# input order that the  sql need 

query = "insert into image (image_data,capture_time) values (%s,%s)"


i = 0
while True :
    
    
    cap = cv2.VideoCapture(url)
    
    ret,frame = cap.read()
    
    if not ret :
        
        cap = cv2.VideoCapture(url)
        
        ret,frame = cap.read()
        
    
    
   
    ret , buffer = cv2.imencode(".jpg",frame)
    
    ##   
    
    if i % 500 == 0:
        
        cur.execute("""
        
        CREATE TABLE IF NOT EXISTS image (
            id INT AUTO_INCREMENT PRIMARY KEY,
            image_data BLOB NOT NULL,
            capture_time DATETIME NOT NULL
        )
    """
        )
        
        cursor.commit()
        
        
        # store into mysql 
        
        capture_time = datetime.now()
        
        print(capture_time)
        
        image_bytes = buffer.tobytes()
        
        
        cur.execute(query,(image_bytes,capture_time))
        
        cursor.commit()
        
        print(" Image data insert to database successfully ")
    
    # we collect 5000 image data 
    if i ==  25000:
        
        print( "  collecting image data is enough ")
        
        break  
    
    i += 1 
    

cursor.close()
cap.release()

cv2.destroyAllWindows() 

In [ ]:
from ultralytics  import YOLO
import cv2 
model = YOLO("yolo12n.pt")

url = "https://cctvn.freeway.gov.tw/abs2mjpg/bmjpg?camera=13380"


while True :
    
    cap = cv2.VideoCapture(url)
    
    ret,frame = cap.read()
    
    if not ret :
        
        cap = cv2.VideoCapture(url)
        ret,frame = cap.read()
        
   

    result = model.predict(frame)[0].boxes.xyxy.
    
    print(result)
    
    
    


0: 448x640 (no detections), 25.1ms
Speed: 2.2ms preprocess, 25.1ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)
ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'oran

KeyboardInterrupt: 

## select image image data from database  

In [ ]:
import mysql.connector as con 

# connect mysql 

cursor = con.connect(
    host = "172.26.176.1",
    user = 'ruizhong' ,
    password = "123456",
    database = "the_volume_of_car"
)



## train model 


    following step

    1 、 100 epoch 

    2 、 try batch 16 、 32 、 64 and 128 

    3 、  


In [ ]:
from  ultralytics import YOLO

model = YOLO("yolo12n.pt") 

# Train the model on the COCO8 dataset for 100 epochs

train_results = model.train(
    data="coco8.yaml",  # Path to dataset configuration file
    epochs=100,  # Number of training epochs
    imgsz=640,  # Image size for training
    device= 0 ,  # Device to run on (e.g., 'cpu', 0, [0,1,2,3])
    batchsize

)



# Evaluate the model's performance on the validation set
metrics = model.val()


In [ ]:
# Fine-tune on your dataset
results = model.train(
    data="/path/to/data.yaml",  # Path to your dataset YAML (from April 19, 2025)
    epochs=50,                  # Adjust based on dataset size
    imgsz=(320,240),                  # Image size for car detection
    batch=16,                   # Adjust for GPU memory (e.g., 8 for 2GB VRAM)
    name="yolov12n_finetuned",  # Output directory
    device=0,                   # GPU device
    patience=20,                # Early stopping
    lr0=0.001,                  # Initial learning rate for fine-tuning
    lrf=0.01,                   # Final learning rate factor
    cos_lr=True,                # Use cosine annealing
    optimizer="AdamW",          # Optimizer (AdamW is robust for YOLOv12)
    mosaic=1.0,                 # Enable mosaic augmentation

)

In [ ]:


'''


Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
  1/50      2.3G    0.04567    0.02345    0.01234         10    320x240
  

Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 2/2 [00:01<00:00,  1.50it/s]
all         50        120      0.750      0.680      0.720      0.450


'''
